### Classification challenge: help bank dig out the reason of unusual high churn rate

In [1]:
# importing libraries
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
tf.__version__

'2.4.1'

#### Part 1：data preprocessing

##### import dataset

In [3]:
dataset = pd.read_csv(r'Churn_Modelling.csv')
# quick look at the data
dataset.head()
# from the data, found that 'RowNumber', 'CustomerId' and 'Surname' are just identifier, do not have impact on whether they leave the bank

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# extract X and Y from dataset
X = dataset.iloc[:,3:-1].values # from fourth column to the second last column, return an array/matrix
y = dataset.iloc[:,-1].values # last column

##### encoding categorical data

In [5]:
# categorical columns："Geography" and "Gender"
# label encoding "Gender"
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2]) # male is 1 female is 0

In [6]:
# on-hot encoding "Geography"
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(),[1])],remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

##### split the dataset into training set and test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

##### feature scaling

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# fit_transform and transform?

#### Part 2: building the ANN

##### initializing the ANN (as a sequence of layers)

In [9]:
# create an object of certain class
ann = tf.keras.models.Sequential()

##### adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
# what is Dense class?
# how do we know how many neurons we want?
# activation function categories:

##### adding the second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

##### adding the output layer

In [12]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
# output layers contains dimensions of output
# in our case, binary outcome, so one neuron, so unit = 1
# sigmoid activation function can map result to 0-1

#### Part 3: training the ANN

##### compiling the ANN

In [13]:
# compile the ANN with optimizer, lost function and metric(accuracy)
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# what is compile?
# optimizer?
# loss function:

##### trainig the ANN on the training set

In [14]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)
# batch_size?

Epoch 1/100
250/250 [==============================] - 1s 578us/step - loss: 0.7011 - accuracy: 0.6083
Epoch 2/100
250/250 [==============================] - 0s 582us/step - loss: 0.5001 - accuracy: 0.7999
Epoch 3/100
250/250 [==============================] - 0s 917us/step - loss: 0.4639 - accuracy: 0.8035
Epoch 4/100
250/250 [==============================] - 0s 536us/step - loss: 0.4300 - accuracy: 0.8182
Epoch 5/100
250/250 [==============================] - 0s 565us/step - loss: 0.4242 - accuracy: 0.8139
Epoch 6/100
250/250 [==============================] - 0s 547us/step - loss: 0.4276 - accuracy: 0.8141
Epoch 7/100
250/250 [==============================] - 0s 561us/step - loss: 0.4174 - accuracy: 0.8219
Epoch 8/100
250/250 [==============================] - 0s 679us/step - loss: 0.3989 - accuracy: 0.8286
Epoch 9/100
250/250 [==============================] - 0s 846us/step - loss: 0.4023 - accuracy: 0.8235
Epoch 10/100
250/250 [==============================] - 0s 746us/step - l

250/250 [==============================] - 0s 763us/step - loss: 0.3312 - accuracy: 0.8621
Epoch 80/100
250/250 [==============================] - 0s 784us/step - loss: 0.3265 - accuracy: 0.8693
Epoch 81/100
250/250 [==============================] - 0s 581us/step - loss: 0.3344 - accuracy: 0.8630
Epoch 82/100
250/250 [==============================] - 0s 619us/step - loss: 0.3341 - accuracy: 0.8652
Epoch 83/100
250/250 [==============================] - 0s 660us/step - loss: 0.3337 - accuracy: 0.8625
Epoch 84/100
250/250 [==============================] - 0s 629us/step - loss: 0.3359 - accuracy: 0.8647
Epoch 85/100
250/250 [==============================] - 0s 619us/step - loss: 0.3341 - accuracy: 0.8623
Epoch 86/100
250/250 [==============================] - 0s 599us/step - loss: 0.3171 - accuracy: 0.8740
Epoch 87/100
250/250 [==============================] - 0s 601us/step - loss: 0.3364 - accuracy: 0.8621
Epoch 88/100
250/250 [==============================] - 0s 641us/step - loss:

#### Part 4: making the predictions and evaluating the model

##### predicting the test set results

In [17]:
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5) 
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


##### making the confusion matrix

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1532   63]
 [ 214  191]]


0.8615

#### predicting the result of a single observation

In [16]:
# France, 600, male, 40, 3, 60000, 2, yes, yes, 50000
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])))
# the probability of this customer leaving the bank is 4.38%
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

[[0.04388708]]
[[False]]
